<a href="https://colab.research.google.com/github/chaelinoh/itb2022/blob/main/%EB%B6%84%EB%A5%98%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import Modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import warnings 

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import svm



%matplotlib inline

plt.rc('font', family="NanumBarunGothic")

df = pd.read_csv('/content/drive/MyDrive/project/health_data.csv')

#데이터 실펴보기

print(df.head())
print(df.describe())

In [ ]:
#데이터타입, null 정보 등 확인
df.info()

In [ ]:
#'수축기혈압','이완기혈압','식전혈당(공복혈당)','트리글리세라이드','HDL콜레스테롤','LDL콜레스테롤'과'총콜레스테롤'의 상관관계 분석을 진행해 보자.
df = df[['수축기혈압','이완기혈압','식전혈당(공복혈당)','트리글리세라이드','HDL콜레스테롤','LDL콜레스테롤','총콜레스테롤']]

In [ ]:
# 결측치 확인 
df.isna().sum()


In [ ]:
#결측치가 존재하는 모든 행 제거

df=df.dropna()
display(df.describe())

In [ ]:
# 한글 폰트 사용 

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
plt.rc('font',family='NanumBarunGothic')

In [ ]:
#타깃 변수인 총콜레스테롤의 정상범주인 120~220을 기준으로 작으면 0 정상은 1 높으면 2로 바꿔준다
def reshape(values):
    if values < 120:
        return 0
    elif 120 <= values <= 220:
        return 1
    else:
        return 2

df['총콜레스테롤'] = df['총콜레스테롤'].apply(reshape)
df

In [ ]:
display(df)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]   #변수 6개
y = df.iloc[:, -1]  #target 변수 총콜레스테롤


#데이터셋을 학습(train)과 테스트(test) 세트로 분할
#random_state : random으로 분할 시 사용되는 난수 seed 숫자

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state=42)

tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

print("학습용 데이터 정확도: {:.3f}".format(tree.score(X_train, y_train)))
print("시험용 데이터 정확도: {:.3f}".format(tree.score(X_test, y_test)))

In [ ]:
#의사결정나무의 깊이를 4로 제한 (과적합 줄임)

tree = DecisionTreeClassifier(max_depth=4, random_state=0)
tree.fit(X_train, y_train)

print("학습용 데이터 정확도: {:.3f}".format(tree.score(X_train, y_train)))
print("시험용 데이터 정확도: {:.3f}".format(tree.score(X_test, y_test)))

#의사결정나무
##주요변수들
- LDL 콜레스테롤 / HDL콜레스테롤 / 트리글리세라이드 가 총콜레스테롤과 가장 연관이 있음을 알 수 있다.
- HDL은 혈관벽에 침착되어 있는 콜레스테롤을 포착해 세포로 전달하므로 높을수록 몸에 좋다.
LDL은 혈관벽에 축적되면서 다양한 심혈관질환을 일으키므로 낮아야 좋다.
- 이를 바탕으로 의사결정나무 결과를 분석해보면 LDL이 134.5 이하인 경우 최종적으로 총콜레스테롤 수치가 정상인 경우가 많고 그 중에서 HDL콜레스테롤이 76.5보다 높은 경우에만 총콜레스테롤 수치가 주의 단계가 되었다.
- 의사결정나무 그래프를 바탕으로 중성지방인 트리글리세라이드가 높을 수록 총콜레스테롤 수치도 높게 나옴을 알 수 있다.

In [ ]:
#  graphviz 라이브러리를 이용하여 의사결정나무 그리기
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(tree, out_file='tree.dot', class_names=['부족','정상','주의'], feature_names=df.columns[:-1], impurity=False, filled=True)


with open("tree.dot") as f:
    dot_graph = f.read()

display(graphviz.Source(dot_graph))